In [2]:
import requests,re,bs4
import sqlite3

In [153]:
url = "http://www.darklyrics.com/lyrics/haken/visions.html"
html = requests.get(url)
soup = bs4.BeautifulSoup(html.content,'html.parser')

In [154]:
metainfo    = [soup.find('h1').text,soup.find('h2').text]
metainfo[0] = metainfo[0].replace(' LYRICS','').capitalize()
metainfo.append(int(re.findall('\(([0-9]{4})\)',metainfo[1])[0]))
metainfo[1] = re.search('"(.*)"',metainfo[1]).groups()[0]
metainfo

['Haken', 'Visions', 2011]

In [155]:
lyrics     = soup.find('div', class_='lyrics')
tagnames   = [child.name for child in list(lyrics.children)]
begins     = [i for i,x in enumerate(tagnames) if x=='h3']
ends       = [i-1 for i in begins][1:] + [len(tagnames)]

In [156]:
class song(object):
    def __init__(self, name,lyrics,artist,genre,album,year):
        self.name   = name
        self.lyrics = lyrics
        self.artist = artist
        self.genre  = genre
        self.album  = album
        self.year   = year

In [234]:
songnames  = [list(lyrics.children)[i].text for i in begins]
songlyrics = [list(lyrics.children)[(x+1):(ends[i]+1)] for i,x in enumerate(begins)]
songlyrics = [[line for line in lyric if type(line) is bs4.element.NavigableString] for lyric in songlyrics]
songlyrics = [' '.join(lyric) for lyric in songlyrics]
print(songlyrics[1])


No disturbance could wake me 
Observers are captivated 
By nocturnal projections 
Of my vivid imagination 
 
Try as I might to recall the events of the night 
Yet the deeper I hunt, the further away they run 
 
Moon rises 
I shiver, confiding in the mirror 
My ghost in the reflection 
Just witnessed the resurrection 
 
Lights flashed before my eyes 
With scenes from someone else's life! 
 
Nothing is quite as it seems 
Something died inside of me 
By powers that be 
A man will live forever 
In paranoia 
 
I believe your conspiracy 
When they're after me 
For what I revealed could change your life 
 
Scenes from someone else's life 
Have they been placed in our minds? 
 
Never have I ever taken a life 
But it seems overnight I've become a killer 
On the run 'cause... 
 
I believe your conspiracy 
When they're after me 
For what I revealed 
 
I believe your conspiracy 
When they're after me 
For what I revealed will change your life 
 
Tonight I hang by a thread 
As the voice in my head

In [ ]:
songs = []
for i,x in enumerate(songnames):
    songs.append(song(x,songlyrics[i],metainfo[0],'Progressive Metal',metainfo[1],metainfo[2]))

In [163]:
songs[1].__dict__.values()

dict_values(['2. Nocturnal Conspiracy', "No disturbance could wake me Observers are captivated By nocturnal projections Of my vivid imagination  Try as I might to recall the events of the night Yet the deeper I hunt, the further away they run  Moon rises I shiver, confiding in the mirror My ghost in the reflection Just witnessed the resurrection  Lights flashed before my eyes With scenes from someone else's life!  Nothing is quite as it seems Something died inside of me By powers that be A man will live forever In paranoia  I believe your conspiracy When they're after me For what I revealed could change your life  Scenes from someone else's life Have they been placed in our minds?  Never have I ever taken a life But it seems overnight I've become a killer On the run 'cause...  I believe your conspiracy When they're after me For what I revealed  I believe your conspiracy When they're after me For what I revealed will change your life  Tonight I hang by a thread As the voice in my head t

## SQL'e yazma

In [217]:
import sqlite3
conn = sqlite3.connect('outputs/lyrics.db')

conn.execute('DROP TABLE lyrics') # daha önce oluşturduğumuz tabloyu sil
cur  = conn.cursor()

cur.execute('''CREATE TABLE lyrics
             (name TEXT, lyrics TEXT, artist TEXT, genre TEXT, album TEXT,year INTEGER)''')
cur.execute('''CREATE UNIQUE INDEX id ON lyrics (name, artist, album)''')

for s in songs:
    # Yazma
    cur.execute("INSERT INTO lyrics VALUES (?,?,?,?,?,?)", (s.name,s.lyrics,s.artist,s.genre,s.album,s.year))

conn.commit()    
conn.close()

In [219]:
import sqlite3
conn = sqlite3.connect('outputs/lyrics.db')
cur  = conn.cursor()

for s in songs:
    # Yazma
    cur.execute("INSERT OR IGNORE INTO lyrics VALUES (?,?,?,?,?,?)", (s.name,s.lyrics,s.artist,s.genre,s.album,s.year))

conn.commit()    
conn.close()

## Album URL üretme

In [176]:
url  = 'http://www.darklyrics.com/d/dreamtheater.html'
soup = bs4.BeautifulSoup(requests.get(url).content,'html.parser')
soup.prettify()

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <title>\n   DREAM THEATER lyrics\n  </title>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <meta content=\'DREAM THEATER lyrics - 154 song lyrics from 16 albums, including "The Astonishing" (2016).\' name="description"/>\n  <meta content="DREAM THEATER, lyrics, discography, albums, songs" name="keywords"/>\n  <link href="../dark.css" rel="stylesheet" type="text/css"/>\n  <script type="text/javascript">\n   var _gaq = _gaq || [];\r\n  _gaq.push([\'_setAccount\', \'UA-20396473-1\']);\r\n  _gaq.push([\'_trackPageview\']);\r\n\r\n  (function() {\r\n    var ga = document.createElement(\'script\'); ga.type = \'text/javascript\'; ga.async = true;\r\n    ga.src = (\'https:\' == document.location.protocol ? \'https://ssl\' : \'http://www\') + \'.google-analytics.com

In [187]:
url_list = [album.find('a').get('href') for album in soup.find_all('div', class_='album')]
url_list = [re.sub('#1|\.\./','',url) for url in url_list]
url_list = ['http://www.darklyrics.com/' + url for url in url_list]
url_list

['http://www.darklyrics.com/lyrics/dreamtheater/whendreamanddayunite.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/imagesandwords.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/awake.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/achangeofseasons.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/fallingintoinfinity.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/scenesfromamemory.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/sixdegreesofinnerturbulence.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/trainofthought.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/octavarium.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/score.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/systematicchaos.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/blackcloudssilverlinings.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/adramaticturnofevents.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/dreamtheater.html

# Darklyrics Crawler
INPUT  : Album URL, genre
OUTPUT : Artist name, album name, genre, release year, lyrics, song name (WRITTEN to SQL)

In [189]:
import requests,re,bs4
import sqlite3

class song(object):
    def __init__(self, name,lyrics,artist,genre,album,year):
        self.name   = name
        self.lyrics = lyrics
        self.artist = artist
        self.genre  = genre
        self.album  = album
        self.year   = year

In [190]:
def album_scraper(url,genre):
    html = requests.get(url)
    soup = bs4.BeautifulSoup(html.content,'html.parser')
    # Meta info extraction
    metainfo    = [soup.find('h1').text,soup.find('h2').text]
    metainfo[0] = metainfo[0].replace(' LYRICS','').capitalize()
    metainfo.append(int(re.findall('\(([0-9]{4})\)',metainfo[1])[0]))
    metainfo[1] = re.search('"(.*)"',metainfo[1]).groups()[0]
    # lyric extraction
    lyrics     = soup.find('div', class_='lyrics')
    tagnames   = [child.name for child in list(lyrics.children)]
    begins     = [i for i,x in enumerate(tagnames) if x=='h3']
    ends       = [i-1 for i in begins][1:] + [len(tagnames)]
    # song name and lyrics
    songnames  = [list(lyrics.children)[i].text for i in begins]
    songlyrics = [list(lyrics.children)[(x+1):(ends[i]+1)] for i,x in enumerate(begins)]
    songlyrics = [[line.strip() for line in lyric if type(line) is bs4.element.NavigableString] for lyric in songlyrics]
    songlyrics = [' '.join(lyric) for lyric in songlyrics]
    #
    # Creation of song and Album (songs list)
    songs = []
    for i,x in enumerate(songnames):
        songs.append(song(x,songlyrics[i],metainfo[0],genre,metainfo[1],metainfo[2]))
        print('Scraped %s of %s, %s' % (x,metainfo[0],metainfo[1]))
    return(songs)

In [215]:
def albumwriter(song_info_list,dbdest):
    conn = sqlite3.connect(dbdest)
    cur  = conn.cursor()

    for s in song_info_list:
        # Yazma
        cur.execute("INSERT INTO lyrics VALUES (?,?,?,?,?,?)", (s.name,s.lyrics,s.artist,s.genre,s.album,s.year))
        print('[LOG] Written songs for %s - %s' % (s.album, s.artist))
    conn.commit()    
    conn.close()

In [206]:
def url_extractor(band_name,genre):
    band_name = band_name.replace(' ','').lower()
    print(band_name)
    first_letter = band_name[0]
    url  = 'http://www.darklyrics.com/%s/%s.html' % (first_letter,band_name)
    soup = bs4.BeautifulSoup(requests.get(url).content,'html.parser')
    url_list = [album.find('a').get('href') for album in soup.find_all('div', class_='album')]
    url_list = [re.sub('#1|\.\./','',url) for url in url_list]
    url_list = ['http://www.darklyrics.com/' + url for url in url_list]
    print('[LOG]: Scraped %i urls for %s' % (len(url_list),band_name))
    return(url_list)

In [199]:
song_list = dlscraper('http://www.darklyrics.com/lyrics/dreamtheater/imagesandwords.html','Progressive Metal')

Scraped 1. Pull Me Under in Dream theater, Images And Words
Scraped 2. Another Day in Dream theater, Images And Words
Scraped 3. Take The Time in Dream theater, Images And Words
Scraped 4. Surrounded in Dream theater, Images And Words
Scraped 5. Metropolis - Part I - "The Miracle And The Sleeper" in Dream theater, Images And Words
Scraped 6. Under A Glass Moon in Dream theater, Images And Words
Scraped 7. Wait For Sleep in Dream theater, Images And Words
Scraped 8. Learning To Live in Dream theater, Images And Words


In [207]:
url_list = url_extractor('Dream Theater','Progressive Metal')
url_list

dreamtheater
[LOG]: Scraped 16 urls for dreamtheater


['http://www.darklyrics.com/lyrics/dreamtheater/whendreamanddayunite.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/imagesandwords.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/awake.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/achangeofseasons.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/fallingintoinfinity.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/scenesfromamemory.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/sixdegreesofinnerturbulence.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/trainofthought.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/octavarium.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/score.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/systematicchaos.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/blackcloudssilverlinings.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/adramaticturnofevents.html',
 'http://www.darklyrics.com/lyrics/dreamtheater/dreamtheater.html

In [210]:
song_lists = [dlscraper(url,'Progressive Metal') for url in url_list]

Scraped 1. A Fortune In Lies in Dream theater, When Dream And Day Unite
Scraped 2. Status Seeker in Dream theater, When Dream And Day Unite
Scraped 3. The Ytse Jam in Dream theater, When Dream And Day Unite
Scraped 4. The Killing Hand in Dream theater, When Dream And Day Unite
Scraped 5. Light Fuse And Get Away in Dream theater, When Dream And Day Unite
Scraped 6. Afterlife in Dream theater, When Dream And Day Unite
Scraped 7. The Ones Who Help To Set The Sun in Dream theater, When Dream And Day Unite
Scraped 8. Only A Matter Of Time in Dream theater, When Dream And Day Unite
Scraped 1. Pull Me Under in Dream theater, Images And Words
Scraped 2. Another Day in Dream theater, Images And Words
Scraped 3. Take The Time in Dream theater, Images And Words
Scraped 4. Surrounded in Dream theater, Images And Words
Scraped 5. Metropolis - Part I - "The Miracle And The Sleeper" in Dream theater, Images And Words
Scraped 6. Under A Glass Moon in Dream theater, Images And Words
Scraped 7. Wait For

Scraped 1. Descent Of The Nomacs in Dream theater, The Astonishing
Scraped 2. Dystopian Overture in Dream theater, The Astonishing
Scraped 3. The Gift Of Music in Dream theater, The Astonishing
Scraped 4. The Answer in Dream theater, The Astonishing
Scraped 5. A Better Life in Dream theater, The Astonishing
Scraped 6. Lord Nafaryus in Dream theater, The Astonishing
Scraped 7. A Savior In The Square in Dream theater, The Astonishing
Scraped 8. When Your Time Has Come in Dream theater, The Astonishing
Scraped 9. Act Of Faythe in Dream theater, The Astonishing
Scraped 10. Three Days in Dream theater, The Astonishing
Scraped 11. The Hovering Sojourn in Dream theater, The Astonishing
Scraped 12. Brother, Can You Hear Me? in Dream theater, The Astonishing
Scraped 13. A Life Left Behind in Dream theater, The Astonishing
Scraped 14. Ravenskill in Dream theater, The Astonishing
Scraped 15. Chosen in Dream theater, The Astonishing
Scraped 16. A Tempting Offer in Dream theater, The Astonishing
Sc

In [220]:
for song_list in song_lists:
    albumwriter(song_list, 'outputs/lyrics.db')

[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for When Dream And Day Unite - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for Images And Words - Dream theater
[LOG] Written songs for 

In [214]:
song_list[0].artist

'Dream theater'

In [221]:
song_list = dlscraper('http://www.darklyrics.com/lyrics/haken/visions.html','Progressive Metal')

Scraped 1. Premonition in Haken, Visions
Scraped 2. Nocturnal Conspiracy in Haken, Visions
Scraped 3. Insomnia in Haken, Visions
Scraped 4. The Mind's Eye in Haken, Visions
Scraped 5. Portals in Haken, Visions
Scraped 6. Shapeshifter in Haken, Visions
Scraped 7. Deathless in Haken, Visions
Scraped 8. Visions in Haken, Visions


In [227]:
song_list[4].lyrics

' '

# Lyrics Crawler (Object) 

In [17]:
import requests,re,bs4
import sqlite3
from time import sleep

class DarkLyricsCrawler(object):
    
    class song(object):
        def __init__(self, name,lyrics,artist,genre,album,year):
            self.name   = name
            self.lyrics = lyrics
            self.artist = artist
            self.genre  = genre
            self.album  = album
            self.year   = year
            
    def url_extractor(self,band_name,genre):
        band_name = band_name.replace(' ','').lower()
        first_letter = band_name[0]
        url  = 'http://www.darklyrics.com/%s/%s.html' % (first_letter,band_name)
        soup = bs4.BeautifulSoup(requests.get(url).content,'html.parser')
        url_list = [album.find('a').get('href') for album in soup.find_all('div', class_='album')]
        url_list = [re.sub('#1|\.\./','',url) for url in url_list]
        url_list = ['http://www.darklyrics.com/' + url for url in url_list]
        print('[LOG]: Scraped %i urls for %s' % (len(url_list),band_name))
        return((url_list,genre))

    def album_scraper(self,url,genre):
        html = requests.get(url)
        soup = bs4.BeautifulSoup(html.content,'html.parser')
        # Meta info extraction
        metainfo    = [soup.find('h1').text,soup.find('h2').text]
        metainfo[0] = metainfo[0].replace(' LYRICS','').capitalize()
        metainfo.append(''.join(re.findall('\(([0-9]{4})\)',metainfo[1])))
        if not metainfo[2]:
            metainfo[2] == None
            metainfo[1] == None
        else:
            metainfo[2] = int(metainfo[2])
            metainfo[1] = re.search('"(.*)"',metainfo[1]).groups()[0]
        # lyric extraction
        lyrics     = soup.find('div', class_='lyrics')
        tagnames   = [child.name for child in list(lyrics.children)]
        begins     = [i for i,x in enumerate(tagnames) if x=='h3']
        ends       = [i-1 for i in begins][1:] + [len(tagnames)]
        # song name and lyrics
        songnames  = [list(lyrics.children)[i].text for i in begins]
        songlyrics = [list(lyrics.children)[(x+1):(ends[i]+1)] for i,x in enumerate(begins)]
        songlyrics = [[line.strip() for line in lyric if type(line) is bs4.element.NavigableString] for lyric in songlyrics]
        songlyrics = [' '.join(lyric) for lyric in songlyrics]
        #
        # Creation of song and Album (songs list)
        songs = []
        for i,x in enumerate(songnames):
            songs.append(self.song(x,songlyrics[i],metainfo[0],genre,metainfo[1],metainfo[2]))
            print('Scraped %s of %s, %s' % (x,metainfo[0],metainfo[1]))
        sleep(3)
        return(songs)

    def db_creator(self, dbpath, dbname):
        conn = sqlite3.connect(dbpath + '/' + dbname)
        cur  = conn.cursor()
        cur.execute('''CREATE TABLE lyrics
                     (name TEXT, lyrics TEXT, artist TEXT, genre TEXT, album TEXT,year INTEGER, source TEXT)''')
        cur.execute('''CREATE UNIQUE INDEX id ON lyrics (name, artist, album)''')

        conn.commit()    
        conn.close()

    def album_writer(self, song_info_list,dbdest):
        conn = sqlite3.connect(dbdest)
        cur  = conn.cursor()

        for s in song_info_list:
            # Yazma
            cur.execute("INSERT OR IGNORE INTO lyrics VALUES (?,?,?,?,?,?,?)", 
                        (s.name,s.lyrics,s.artist,s.genre,s.album,s.year,'Darklyrics'))
            print('[LOG] Written songs for %s - %s' % (s.album, s.artist))
        conn.commit()    
        conn.close()

In [18]:
dc = DarkLyricsCrawler()

In [3]:
band_list  = [('Control Denied','Thrash Metal'), ('Leprous','Progressive Metal')]
album_urls = [dc.url_extractor(band,genre) for band,genre in band_list]

[LOG]: Scraped 1 urls for controldenied
[LOG]: Scraped 6 urls for leprous


In [4]:
album_urls

[(['http://www.darklyrics.com/lyrics/controldenied/thefragileartofexistence.html'],
  'Thrash Metal'),
 (['http://www.darklyrics.com/lyrics/leprous/aeolia.html',
   'http://www.darklyrics.com/lyrics/leprous/tallpoppysyndrome.html',
   'http://www.darklyrics.com/lyrics/leprous/bilateral.html',
   'http://www.darklyrics.com/lyrics/leprous/coal.html',
   'http://www.darklyrics.com/lyrics/leprous/thecongregation.html',
   'http://www.darklyrics.com/lyrics/leprous/malina.html'],
  'Progressive Metal')]

In [5]:
albums = []
for urls, genre in album_urls:
    for url in urls:
        albums.append(dc.album_scraper(url, genre))

Scraped 1. Consumed of Control denied, The Fragile Art Of Existence
Scraped 2. Breaking The Broken of Control denied, The Fragile Art Of Existence
Scraped 3. Expect The Unexpected of Control denied, The Fragile Art Of Existence
Scraped 4. What If...? of Control denied, The Fragile Art Of Existence
Scraped 5. When The Link Becomes Missing of Control denied, The Fragile Art Of Existence
Scraped 6. Believe of Control denied, The Fragile Art Of Existence
Scraped 7. Cut Down of Control denied, The Fragile Art Of Existence
Scraped 8. The Fragile Art Of Existence of Control denied, The Fragile Art Of Existence
Scraped 1. Disclosure of Leprous, Aeolia
Scraped 2. Black Stains of Leprous, Aeolia
Scraped 3. Last Word of Leprous, Aeolia
Scraped 4. Aeolus Shadow of Leprous, Aeolia
Scraped 5. The Great Beast of Leprous, Aeolia
Scraped 6. Indecisive of Leprous, Aeolia
Scraped 7. Close Your Heart of Leprous, Aeolia
Scraped 8. Deformed Beauty of Leprous, Aeolia
Scraped 9. Eye Of The Storm of Leprous, A

In [15]:
albums[1][3].lyrics

'I fear my own shadow In the city smoke Like a wave’s undertow The silence awoke  The city lights are blending My eyes and my soul Differs not day from night Entering black hole  As the wind sweeps the city-sky Aeolus casts his shadow Metropolis at his feet As noise explodes to silence  As total chaos is unleashed The storm eats the city heart Pandemonium is released Damnation of true art  No escaping the truth The gardens all dead No more light to inhale The beast have been fed  I feel my own shadow Creeping up on me Like a wave’s undertow I am not to be  '

In [20]:
dc.album_writer(albums[1],'output/lyrics.db')

[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous
[LOG] Written songs for Aeolia - Leprous


# azlyrics.com 

In [9]:
import requests,re,bs4
import sqlite3
from time import sleep
from random import uniform, randint

url = 'https://raw.githubusercontent.com/asuerdem/SMM/master/1_Crawler/1_newscrawler/user_agent_list.txt'
useragents = requests.get(url)

In [10]:
useragents = useragents.text.split('\n')

In [14]:
from ghost import Ghost
ghost = Ghost()

url = 'https://www.azlyrics.com/f/faithnomore.html'

with ghost.start() as session:
    page, extra_resources = session.open(url)
    assert page.http_status == 200 and 'jeanphix' in page.content


AttributeError: 'NoneType' object has no attribute 'http_status'

In [13]:
useragent = useragents[randint(0,len(useragents) - 1)]
headers = { 'User-Agent': useragent }
url = 'https://www.azlyrics.com/a/adele.html'
html = requests.get(url, headers=headers)
soup = bs4.BeautifulSoup(html.content,'html.parser')

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [6]:
artist_info = soup.find('h1').text
artist_info = artist_info.replace(' Lyrics', '')
artist_info

'Adele'

In [7]:
albums_info = []
for info in soup.find_all('div', class_='album'):
    album_name  = info.text
    release_year = ''.join(re.findall('\(([0-9]+)\)',album_name))
    release_year = int(release_year) if release_year else None
    album_name = ''.join(re.findall(':\s(.*)\s\(',album_name))
    if not album_name: album_name = 'Other Songs' 
    albums_info.append({'Album': album_name, 'Year': release_year})
albums_info

[{'Album': '"19"', 'Year': 2008},
 {'Album': '"21"', 'Year': 2011},
 {'Album': '"25"', 'Year': 2015},
 {'Album': 'Other Songs', 'Year': None}]

In [115]:
album_lyrics = soup.find('div', attrs = {'id':'listAlbum'})
# album_lyrics = album_lyrics.find('a', attrs = {'id':'listAlbum'}0)
album_lyrics = [part for part in list(album_lyrics.children) if part!= '\n']
begins = [i for i, part in enumerate(album_lyrics) if part.name == 'div' and part.has_attr('class')]
ends   = [i-1 for i in begins][1:] + [len(album_lyrics)]
albums = [album_lyrics[begins[i]:ends[i]] for i in range(len(begins))]

In [230]:
def extract_lyrics(url):    
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
    html = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(html.content,'html.parser')

    lyrics = soup.select('div[class*="text-center"]')
    ind = [len(part.find_all('br')) for part in lyrics]
    ind = [i for i,v in enumerate(ind) if v == max(ind)][0]
    # aa = lyrics[ind].findChild('div', attrs={'class':'ringtone'})
    lyrics = lyrics[ind].findChildren('div')
    ind = [len(part.find_all('br')) for part in lyrics]
    ind = [i for i,v in enumerate(ind) if v == max(ind)][0]
    return(lyrics[ind].text)

In [241]:
artist_info = soup.find('h1').text
artist_info = artist_info.replace(' Lyrics', '')
genre       = 'Pop'
# albums_info = []
songs       = []
for album in albums:
    release_year = ''.join(re.findall('\(([0-9]+)\)',album[0].text))
    release_year = int(release_year) if release_year else None
    album_name = ''.join(re.findall(':\s(.*)\s\(',album[0].text))
    if not album_name: album_name = 'Other Songs' 
    # albums_info.append({'Album': album_name, 'Year': release_year})
    for song in album:
        if song.name == 'a':
            songname = song.text
            songlink = song.attrs['href']
            songlink = 'https://www.azlyrics.com/' + songlink.replace('../','')
            lyrics   = extract_lyrics(songlink)
            sleep(uniform(.5,1))
            songs.append({'Album': album_name, 'Year': release_year, 'Name': songname,
                          'Link': songlink, 'Artist':artist_info,'Lyrics':lyrics,'Genre':genre})
    sleep(uniform(.5,1))

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [234]:
from time import sleep
for song in songs[:4]:
    sleep(.4)
    print(song['Lyrics'])



Daydreamer
Sitting on the sea
Soaking up the sun
He is a real lover
Of making up the past
And feeling up his girl
Like he's never felt a figure before

A jaw-dropper
Looks good when he walks
Is the subject of their talk
He would be hard to chase
But good to catch
And he could change the world
With his hands behind his back, oh

You can find him sittin' on your doorstep
Waiting for a surprise
And he will feel like he's been there for hours
And you can tell that he'll be there for life

Daydreamer
With eyes that make you melt
He lends his coat for shelter
Plus he's there for you
When he shouldn't be
But he stays all the same
Waits for you
Then sees you through

There's no way I could describe him
What I've said is just what I'm hoping for

But I will find him sittin' on my doorstep
Waiting for a surprise
And he will feel like he's been there for hours
And I can tell that he'll be there for life
And I can tell that he'll be there for life



Wait, do you see my heart on my sleeve?
It's 

In [238]:
def db_creator(dbpath, dbname):
    conn = sqlite3.connect(dbpath + '/' + dbname)
    cur  = conn.cursor()
    cur.execute('''CREATE TABLE lyrics
                 (name TEXT, lyrics TEXT, artist TEXT, genre TEXT, album TEXT,year INTEGER, source TEXT)''')
    cur.execute('''CREATE UNIQUE INDEX id ON lyrics (name, artist, album)''')

    conn.commit()    
    conn.close()

def album_writer(song_info_list,dbdest):
    conn = sqlite3.connect(dbdest)
    cur  = conn.cursor()

    for s in song_info_list:
        # Yazma
        cur.execute("INSERT OR IGNORE INTO lyrics VALUES (?,?,?,?,?,?,?)", 
                    (s['Name'],s['Lyrics'],s['Artist'],s['Genre'],s['Album'],s['Year'],'azlyrics'))
        print('[LOG] Written songs for %s - %s' % (s.album, s.artist))
    conn.commit()    
    conn.close()

In [239]:
album_writer(songs,'output/lyrics.db')

In [240]:
songs

[]